# Cookbook for the course
In this Notebook, you'll find snippets of code that you'll use again and again.
A semicolon (";") is used at the end of lines to supress the output and keep the Notebook compact.

This cookbook is focused on data cleaning, plotting and supervised learning. Unsupervised learning (clustering), collaborative filtering and text mining can be found in the respective Notebooks.


In [2]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import math

## Reading data and summarizing

In [3]:
df = pd.read_csv("googleplay.csv")
df.describe(); #get some summary statistics
df['rating'].value_counts(); #get a simple table of this variable
df.head(1) #get the first rows


app category  rating  reviews  size_mb   installs  type  \
0  Samsung Calculator    TOOLS     4.4     9602      2.5  100000000  Free   

   price content_rating  genre       updated   version     android  
0    0.0       Everyone  Tools  July 5, 2018  6.0.61.5  7.0 and up

## Data cleaning and pre-processing

In [4]:
#Subset the dataframe with a list of colums. Note the double [[]]
df_a = df[['app', 'size_mb']] 

#Get everything up to reviews. .loc works just like Python slicing of lists: first rows, then columns.
df_b = df.loc[:,:'category'] 

#Merge two data sets using a common identifier
df_ab = pd.merge(df_a, df_b, on='app') 
df_ab.head(1)

app  size_mb category
0  Samsung Calculator      2.5    TOOLS

In [5]:
#Selecting rows based on a condition
df_subset = df[df['rating'] > 4] #gets all the apps with ratings higher than 4
df_subset = df[(df['rating'] > 4) & (df['type'] == 'Free')] #gets all the free apps rated higher than 4

#Drop all rows with missing values.
#Don't use this if there are a lot. Instead look on the internet for information on imputing missing values.
df = df.dropna() #I'm actually doing this to the df because I need the df to be NA-free later on

#Replacing all values (e.g. implausible/impossible) with another value
#Define a function, then apply that function to all relevant items
def above_5(x):
    if(x > 5.0): 
        return float('NaN')
    else: 
        return x
df['rating'] = df['rating'].apply(above_5) #apply the function to all items in the column

#Log transforming the skewed variable 'installs'
#Apply the log10 function from the math library to all items in reviews
df['installs_log'] = df['installs'].apply(math.log10)
df.head(1)

app category  rating  reviews  size_mb   installs  type  \
0  Samsung Calculator    TOOLS     4.4     9602      2.5  100000000  Free   

   price content_rating  genre       updated   version     android  \
0    0.0       Everyone  Tools  July 5, 2018  6.0.61.5  7.0 and up   

   installs_log  
0           8.0

### Dummy variables

In [6]:
#Creating dummy variables of qualitative variables
dummies = pd.get_dummies(df['content_rating'])
dummies.head(1)

Everyone  Everyone 10+  Mature 17+  Teen
0         1             0           0     0

In [7]:
#Adding dummies to the df by concatenating variables to dataframe
df = pd.concat([df, dummies], axis=1) #the axis=1 means: add it to the columns (axis=0 is rows)
df.head(1)

app category  rating  reviews  size_mb   installs  type  \
0  Samsung Calculator    TOOLS     4.4     9602      2.5  100000000  Free   

   price content_rating  genre       updated   version     android  \
0    0.0       Everyone  Tools  July 5, 2018  6.0.61.5  7.0 and up   

   installs_log  Everyone  Everyone 10+  Mature 17+  Teen  
0           8.0         1             0           0     0

## Pivot tables
With pivot tables, you can easily create summaries of dataframes.

In [9]:
#Get a table of mean rating by genre
pd.pivot_table(data=df, index='genre', values='rating');

#Get a table of mean rating by genre x content_rating
pd.pivot_table(data=df, index='genre', columns='content_rating' ,values='rating');

#The mean is the standard aggregating function. However, you can use other functions like summation.
pd.pivot_table(data=df, index='genre', columns='content_rating' ,values='reviews', aggfunc='sum');

## Plotting
Some useful plots. These are all commented out because Seaborn ignores the semicolons and plots the figure anyway.

In [50]:
#Distribution
#sns.distplot(df['size_mb']) #histogram with density plot

#Qualitative
#sns.countplot(x='type', data=df) #create a bar graph with counts of variable 'type'

#Qualitative x qualitative plots
#sns.countplot(x='type', hue='content_rating', data=df) #the same bar graph, using color for content_rating

#Qualitative x quantitative plots. These can also be called as sns.catplot, with kind='swarm', 'box', etc.
#sns.swarmplot(x='rating', y='type', data=df) #plot the data as points, split out by app type (free/paid)
#sns.swarmplot(x='rating', y='type', hue='content_rating', data=df) #same, but use color for content_rating
#sns.boxplot(x='rating', y='type', data=df) #box plots 
#sns.violinplot(x='rating', y='type', data=df) #violin plots

#Quantitative x quantitative
#sns.pairplot(df) #scatter plot matrix
#sns.scatterplot(x='price', y='rating', data=df) #single scatter plot
#sns.scatterplot(x='price', y='rating', hue='content_rating', data=df) #same, but use color  for content_rating
#sns.lmplot(x='price', y='rating', data=df) #a linear regression plot

## Supervised learning

The particular functions are different for each algorithm. However, there are many similarities in terms of code. Here, I will only show linear regression (regression) and k-learn (classification).

### Linear regression

In [55]:
from sklearn.linear_model import LinearRegression #the model
from sklearn.model_selection import train_test_split #the function to split the data

#Setting up the data and model
X = df[['installs_log', 'size_mb']] #selecting the variables to go into my X matrix
y = df['rating'] #creating the y vector

#Split the data. test_size = 0.3, so I'm splitting the data into 70% training data and 30% test data
#I'm using the subscript _l to indicate it's linear regression (using the same variable names in next block)
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(X, y, test_size=0.3) #split the data and store it
# The following line would do the same but would generate the same split every time (use any number for random_state)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) 

lm = LinearRegression(); #create the lm object
lm.fit(X_train_l, y_train_l); #fit the model

### k-nearest neighbor

In [56]:
from sklearn.neighbors import KNeighborsClassifier #the object class we need
from sklearn.preprocessing import normalize #get the function needed to normalize our data.

#Setting up the data and model
X = df[['installs', 'rating']] #selecting the variables to go into my X matrix
X = normalize(X) #normalize the matrix to put everything on the same scale
y = df['type'] #creating the y vector. Type = 'paid' or 'free'

#Split the data. test_size = 0.3, so I'm splitting the data into 70% training data and 30% test data
#I'm using the subscript _k to indicate it's linear regression (using the same variable names in next block)
X_train_k, X_test_k, y_train_k, y_test_k = train_test_split(X, y, test_size=0.3) #split the data and store it

knn = KNeighborsClassifier(n_neighbors=3) #create a KNN-classifier with 2 neighbors
knn = knn.fit(X_train_k, y_train_k)

## Evaluating supervised learning

### Regression

In [58]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_test_p = lm.predict(X_test_l) #get the predicted scores on the test set
resid = y_test_p - y_test_l #residuals of the test set

#R^2: proportion of variance explained
r2_score(y_test_l, y_test_p);

#RMSE: root mean square error: typical error the model makes
math.sqrt(mean_squared_error(y_test_l, y_test_p));

#Plotting the results, again commented out
#sns.scatterplot(y_test, y_test_p) #plotting the predicted scores against the actual scores
#sns.scatterplot(y_test, resid) #plotting the residuals against the actual scores


### Classification

In [59]:
#Get the accuracy of the model (proportion correctly predicted cases)
knn.score(X_test, y_test);

#Get the confusion matrix, from which we can calculate precision and recall for each outcome
from sklearn.metrics import confusion_matrix

y_test_p = knn.predict(X_test_k) #the predicted values
cm = confusion_matrix(y_test_k, y_test_p) #creates a "confusion matrix"
conf_matrix = pd.DataFrame(cm, index=['Free', 'Paid'], columns = ['Free', 'Paid'])